<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Using-bigger-data" data-toc-modified-id="Using-bigger-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Using bigger data</a></span><ul class="toc-item"><li><span><a href="#Training-data" data-toc-modified-id="Training-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Training data</a></span></li></ul></li></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pyspellchecker import SpellChecker
from pyspellchecker import utils

train, test = utils.get_evaluation(data_path='../data/evaluation.txt')

X_tr = [x['original'] for x in train]
y_tr = [x['corrected'] for x in train]
X_te = [x['original'] for x in test]
y_te = [x['corrected'] for x in test]

train_data = [' '.join(x) for x in y_tr]

In [3]:
speller = SpellChecker()

In [4]:
speller.fit(train_data)

In [5]:
speller.transform('the maan went to the store')

['the', 'man', 'went', 'to', 'the', 'store']

In [6]:
import re
token_pattern=r"(?u)\b\w+|\?"
token_pattern = re.compile(token_pattern)
token_pattern.findall('red shirtt? , ! I love it ? ')

['red', 'shirtt', '?', 'I', 'love', 'it', '?']

In [7]:
%%time 
speller.transform('red shirtt')

CPU times: user 124 µs, sys: 1 µs, total: 125 µs
Wall time: 128 µs


['red', 'shirtt']

### Using bigger data

#### Training data


The dataset used to train the model comes from  https://jmcauley.ucsd.edu/data/amazon/

The `Video_Games_5.json` can be downloaded from 
http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Video_Games_5.json.gz

In [8]:
import pandas as pd
from os.path import expanduser
home = expanduser("~")

In [9]:
dataset_path = os.path.join(home,'Datasets/amazon_product_data/reviews_Video_Games_5.json')
df = pd.read_json(dataset_path, lines=True)

In [10]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2HD75EMZR8QLN,0700099867,123,"[8, 12]",Installing the game was a struggle (because of...,1,Pay to unlock content? I don't think so.,1341792000,"07 9, 2012"
1,A3UR8NLLY1ZHCX,0700099867,"Alejandro Henao ""Electronic Junky""","[0, 0]",If you like rally cars get this game you will ...,4,Good rally game,1372550400,"06 30, 2013"
2,A1INA0F5CWW3J4,0700099867,"Amazon Shopper ""Mr.Repsol""","[0, 0]",1st shipment received a book instead of the ga...,1,Wrong key,1403913600,"06 28, 2014"
3,A1DLMTOTHQ4AST,0700099867,ampgreen,"[7, 10]","I got this version instead of the PS3 version,...",3,"awesome game, if it did not crash frequently !!",1315958400,"09 14, 2011"
4,A361M14PU2GUEG,0700099867,"Angry Ryan ""Ryan A. Forrest""","[2, 2]",I had Dirt 2 on Xbox 360 and it was an okay ga...,4,DIRT 3,1308009600,"06 14, 2011"


In [11]:
train_data = list(df['reviewText'])

In [12]:
train_data[3][0:20]

'I got this version i'

In [13]:
speller = SpellChecker()

In [14]:
speller.fit(train_data)

In [15]:
%%timeit
speller.transform('xboj 360 colector edition')

9.44 ms ± 350 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%%timeit
speller.transform('pssp blac edition')

431 ms ± 31.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit
speller.transform('new pleystation gaime batmen')

1.91 s ± 38.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
'batmen' in speller.vocabulary

False

In [19]:
speller.unigram_freq_dict['batmen']

1

Because sometimes very infrequent errors are inside the vocabulary we might want to remove them

In [20]:
speller.filter_vocabulary(min_freq=5)

In [21]:
'batmen' in speller.vocabulary

False

In [22]:
speller.transform('new pleystation gaime batmen')

['new', 'playstation', 'game', 'batman']